## RF

----

### 1. Import Modules

In [28]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load data and peak sheet

In [29]:
home = 'data/' 
file = 'MTBLS93.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 2139 TOTAL PEAKS: 202
Done!


### 3. Extract X & Y

In [30]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)  

### 4. Hyperparameters optimisation

In [31]:
# param_dict
depth = list(range(1,11))
feat = list(range(1,11))
param_dict = dict(max_depth=depth, max_features=feat, criterion='gini', min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, n_estimators=10)


# Initalise
cv = cb.cross_val.kfold(model=cb.model.RF,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=1)                                

# Run and plot
cv.run()  
cv.plot(metric='auc', ci=95, scale=1.3, color_beta=5)

Number of cores set to: 8
Running ...


100%|██████████| 100/100 [00:11<00:00, 11.73it/s]


Time taken: 0.21 minutes with 8 cores
Done!


Loading BokehJS ...

### 6. Get Model and Evaluate

In [32]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.RF(max_depth=4, max_features=10, criterion='gini', min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, n_estimators=10)
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate Ypred against Ytest
model.evaluate(cutoffscore=0.5, plot_median=False) 

Loading BokehJS ...

In [33]:
# Bootstrap evaluate

model.booteval(XTknn, Y, cutoffscore=0.5, bootnum=100) 

100%|██████████| 100/100 [00:07<00:00, 13.12it/s]


Loading BokehJS ...

### 7. Save table

In [34]:
home = 'tables/'
file = 'RF_MTBLS93_NH.xlsx'

model.save_table(home + file)

Done! Saved table as tables/RF_MTBLS93_NH.xlsx
